# side model feature selection

In [10]:
import pandas as pd

df_features = pd.read_parquet("data/features_p009.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(20650, 12762)


4h_dft_dom_cycle_ddt_lag19    2963
4h_dft_dom_cycle_dt_lag19     2956
4h_dft_dom_cycle_ddt_lag18    2956
4h_dft_dom_cycle_lag19        2945
4h_dft_dom_cycle_dt_lag18     2945
                              ... 
p009_conv_16                     0
p009_conv_15                     0
p009_conv_14                     0
p009_conv_13                     0
p009_ac_32                       0
Length: 12762, dtype: int64

In [11]:
import numpy as np
from jesse import utils

label = np.load("data/label_side.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label["side_label"] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-01 21:59:00,7231.61,7165.23,7231.61,7101.0,48155.319,-1.0


In [12]:
side_features = df_features.iloc[2965:]
side_label = df_label.iloc[2965:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

(17685, 12762)
(17685,)


/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_20508/2385448748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


p018_ac_0                   0
4h_williams_r_ddt_lag12     0
4h_williams_r_ddt_lag14     0
4h_williams_r_ddt_lag15     0
4h_williams_r_ddt_lag16     0
                           ..
p0045_ac_2                  0
p0045_ac_3                  0
p0045_ac_4                  0
p0045_ac_5                  0
15m_williams_r_ddt_lag19    0
Length: 12762, dtype: int64

In [13]:
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector()
selector.fit(side_features, side_label)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: p009_williams_r (最大F值: 9161.2720)
➤ 计算特征冗余度...
➤ 总计选择2552个特征 (已选择1个，还需选择2551个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2551/2551 [11:49<00:00,  3.60特征/s]



✅ 特征选择完成：从12762个特征中选择了2552个，舍弃了10210个
✅ 选择的特征: ['p009_williams_r', 'p009_mod_rsi_ddt_lag1', 'p0045_comb_spectrum_pwr_20', 'p009_acc_swing_index_dt', 'p018_acc_swing_index_dt', 'p0045_williams_r', 'p009_fisher_dt', '1h_williams_r', 'p009_trendflex_dt', 'p009_williams_r_dt', 'p009_williams_r_lag1', '15m_williams_r', 'p018_williams_r_dt', 'p009_fisher', 'p009_voss', 'p018_forecast_oscillator', '15m_acc_swing_index_dt', '4h_williams_r', 'p0045_trendflex', 'p009_roofing_filter_ddt', 'p0045_williams_r_lag1', 'p009_vwap_dt', 'p0045_fisher', 'p0045_acc_swing_index_dt', 'p0045_evenbetter_sinewave_long', 'p009_natr_dt', 'p018_fisher_ddt', 'p018_williams_r', 'p018_natr_dt', 'p009_stc_dt', '1h_acc_swing_index_dt', 'p009_evenbetter_sinewave_short', '4h_acc_swing_index_dt', 'p009_stc', 'p009_fisher_dt_lag1', 'p009_voss_dt', 'p0045_trendflex_lag1', 'p018_trendflex_ddt', 'p0045_evenbetter_sinewave_short', 'p009_acc_swing_index_dt_lag1', 'p0045_williams_r_lag2', 'p018_voss_dt', 'p009_trendflex_ddt', '

In [14]:
side_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
side_res

p009_williams_r                           9.161272e+03
p0045_williams_r                          5.696091e+03
p009_fisher_dt                            5.044318e+03
p018_acc_swing_index_dt                   5.019729e+03
p009_trendflex_dt                         4.563335e+03
                                              ...     
p009_evenbetter_sinewave_long_dt_lag19    1.122010e-06
p009_comb_spectrum_dom_cycle_ddt_lag9     9.765877e-07
1h_forecast_oscillator_lag18              7.893519e-07
p018_acc_swing_index_ddt_lag12            6.987570e-07
p0045_williams_r_ddt_lag15                6.657697e-07
Length: 12762, dtype: float64

In [15]:
import json

with open(f"data/side_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(side_res.to_dict(), f, indent=4)

In [16]:
import json

import pandas as pd

with open(f"data/side_features_FCQSelector.json", "r") as f:
    side_res = pd.Series(json.load(f))

In [17]:
RANGE_BAR_SHORT_TERM = "p0045"
RANGE_BAR_MEDIUM_TERM = "p009"
RANGE_BAR_LONG_TERM = "p018"

DOLLAR_BAR_SHORT_TERM = "15m"
DOLLAR_BAR_MEDIUM_TERM = "1h"
DOLLAR_BAR_LONG_TERM = "4h"

feature_info = {
    "side": {
        RANGE_BAR_SHORT_TERM: [],
        RANGE_BAR_MEDIUM_TERM: [],
        RANGE_BAR_LONG_TERM: [],
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
    },
    "meta": {
        RANGE_BAR_SHORT_TERM: [],
        RANGE_BAR_MEDIUM_TERM: [],
        RANGE_BAR_LONG_TERM: [],
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
        "model_res": [],
    },
}

for k, v in side_res[: len(side_res) // 5].to_dict().items():
    if k.startswith(f"{RANGE_BAR_SHORT_TERM}_"):
        feature_info["side"][RANGE_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{RANGE_BAR_MEDIUM_TERM}_"):
        feature_info["side"][RANGE_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{RANGE_BAR_LONG_TERM}_"):
        feature_info["side"][RANGE_BAR_LONG_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["side"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["side"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["side"][DOLLAR_BAR_LONG_TERM].append(k)

feature_info["side"][RANGE_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][RANGE_BAR_SHORT_TERM])),
)
feature_info["side"][RANGE_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][RANGE_BAR_MEDIUM_TERM])),
)
feature_info["side"][RANGE_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][RANGE_BAR_LONG_TERM])),
)
feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][RANGE_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][RANGE_BAR_SHORT_TERM])),
)
feature_info["meta"][RANGE_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][RANGE_BAR_MEDIUM_TERM])),
)
feature_info["meta"][RANGE_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][RANGE_BAR_LONG_TERM])),
)
feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("custom_indicators/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [21]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_p009.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob

assert df_features.shape[0] == meta_label.shape[0]
assert "side_model_res" in df_features.columns

df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag19    2963
4h_dft_dom_cycle_dt_lag19     2956
4h_dft_dom_cycle_ddt_lag18    2956
4h_dft_dom_cycle_lag19        2945
4h_dft_dom_cycle_dt_lag18     2945
                              ... 
p009_comb_spectrum_pwr_2         0
p009_comb_spectrum_pwr_3         0
p009_comb_spectrum_pwr_4         0
p009_comb_spectrum_pwr_5         0
side_model_res                   0
Length: 12763, dtype: int64

In [22]:
meta_features = df_features[2965:]
meta_label = meta_label[2965:]

meta_label = pd.Series(meta_label[:, 6].astype(int), index=meta_features.index)

# meta_features = meta_features[meta_label["ret"].notna()]
# meta_label = meta_label[meta_label["ret"].notna()]["bin"]

print(meta_features.shape)
print(meta_label.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(17685, 12763)
(17685,)


p018_ac_0                  0
4h_williams_r_ddt_lag13    0
4h_williams_r_ddt_lag15    0
4h_williams_r_ddt_lag16    0
4h_williams_r_ddt_lag17    0
                          ..
p0045_ac_2                 0
p0045_ac_3                 0
p0045_ac_4                 0
p0045_ac_5                 0
side_model_res             0
Length: 12763, dtype: int64

In [23]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.fcq_selector import (
    FCQSelector,
)

selector = FCQSelector()
selector.fit(meta_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: p018_natr_dt (最大F值: 105.0261)
➤ 计算特征冗余度...
➤ 总计选择2552个特征 (已选择1个，还需选择2551个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2551/2551 [12:21<00:00,  3.44特征/s]



✅ 特征选择完成：从12763个特征中选择了2552个，舍弃了10211个
✅ 选择的特征: ['p018_natr_dt', '15m_acc_swing_index_ddt_lag4', '1h_pfe_ddt_lag5', 'p018_vwap_lag19', 'p009_conv_3', 'p018_adaptive_bp_dt_lag1', '1h_ac_14', 'p009_comb_spectrum_dom_cycle_dt_lag6', '4h_evenbetter_sinewave_short_ddt_lag6', '15m_mod_stochastic_dt_lag19', 'p009_comb_spectrum_dom_cycle_ddt_lag8', '15m_hurst_coef_slow_ddt_lag4', '4h_trendflex_ddt_lag19', 'p0045_stc_ddt_lag5', '4h_comb_spectrum_dom_cycle_ddt_lag12', '1h_evenbetter_sinewave_short_ddt_lag8', 'p0045_vwap_ddt_lag9', 'p018_comb_spectrum_dom_cycle_ddt_lag16', 'p009_mod_rsi_ddt_lag16', 'p009_hurst_coef_fast_lag7', 'p009_ac_0', 'p009_pfe_lag18', 'p018_natr_ddt', 'p018_ac_45', 'p0045_evenbetter_sinewave_short_ddt_lag9', '4h_hurst_coef_fast_ddt_lag11', 'p0045_dual_diff_ddt', '4h_vwap_ddt_lag6', 'p018_vwap_ddt_lag12', 'p009_acc_swing_index_lag19', 'p0045_natr_dt_lag12', 'p009_comb_spectrum_dom_cycle_ddt_lag11', '1h_homodyne_lag10', 'p0045_vwap_ddt', 'p009_conv_0', '4h_williams_r_ddt_lag2

In [24]:
import json

with open(f"data/meta_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(meta_res.to_dict(), f, indent=4)

In [25]:
import json

import pandas as pd

with open("data/meta_features_FCQSelector.json", "r") as f:
    meta_res = pd.Series(json.load(f))

In [26]:
meta_res.to_dict()["side_model_res"]

10.927161630950247

In [27]:
from pathlib import Path

feature_info = Path("custom_indicators/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

RANGE_BAR_SHORT_TERM = "p0045"
RANGE_BAR_MEDIUM_TERM = "p009"
RANGE_BAR_LONG_TERM = "p018"

DOLLAR_BAR_SHORT_TERM = "15m"
DOLLAR_BAR_MEDIUM_TERM = "1h"
DOLLAR_BAR_LONG_TERM = "4h"

feature_info["meta"][RANGE_BAR_SHORT_TERM] = []
feature_info["meta"][RANGE_BAR_MEDIUM_TERM] = []
feature_info["meta"][RANGE_BAR_LONG_TERM] = []
feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = []
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = []
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = []
feature_info["meta"]["model_res"] = []

for k, v in meta_res[: len(meta_res) // 5].to_dict().items():
    if k.startswith(f"{RANGE_BAR_SHORT_TERM}_"):
        feature_info["meta"][RANGE_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{RANGE_BAR_MEDIUM_TERM}_"):
        feature_info["meta"][RANGE_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{RANGE_BAR_LONG_TERM}_"):
        feature_info["meta"][RANGE_BAR_LONG_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        feature_info["meta"]["model_res"].append(k)

feature_info["side"][RANGE_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][RANGE_BAR_SHORT_TERM])),
)
feature_info["side"][RANGE_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][RANGE_BAR_MEDIUM_TERM])),
)
feature_info["side"][RANGE_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][RANGE_BAR_LONG_TERM])),
)
feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][RANGE_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][RANGE_BAR_SHORT_TERM])),
)
feature_info["meta"][RANGE_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][RANGE_BAR_MEDIUM_TERM])),
)
feature_info["meta"][RANGE_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][RANGE_BAR_LONG_TERM])),
)
feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("custom_indicators/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)